## Digital Design Brief - Jupyter Notebooks

### Virtual Environment Setup
To ensure you're using the virtual environment, you'll need to create a virtual environment, activate it, and the create a kernel using the virtual environment.  You can then launch notebooks from within the environment.
```python
python -m venv .venv
.venv/Scripts/activate
pip install git+https://github.com/arup-group/ddbpy_auth.git
pip install git+https://github.com/Ash-Kulkarni/pyddb.git
ipython kernel install --user --name=venv
jupyter notebook
```
To run these notebooks after this initial installation, the run the following commands to activate the virtual environment and launch the notebooks:
```python
.venv/Scripts/activate
jupyter notebook
```
Within the notebook, you can navigate to the `kernel` tab, and select it.

### Imports
First you should import the necessary libraries.
This will likely prompt you for Arup authentication.

In [1]:
import pyddb
import asyncio

Using default account: Ash.Kulkarni@arup.com


### Getting Started
All python scripts using the `pyddb` library can be executed in a notebook with only a couple of extra lines.

Notebooks have an event loop running in the background, so we can just nest another as a workaround.

In [2]:
import nest_asyncio
nest_asyncio.apply()

### Asynchronous functions
This library uses `asyncio` and `aiohttp`.
You can use the type hints and doc strings for additional information as necessary.

### Initialising the DDB client
The following function shows a typical example of initialising the DDB Client object and setting the environment you're looking to work in.


In [3]:
async def main():
    ddb = pyddb.DDB(url=pyddb.BaseURL.sandbox)
    projects = await ddb.get_projects()
    return projects

asyncio.run(main())

['Name: ARK PIONEER ALL THROUGH FREE S, Project Number: 24700900, ID: 1c1c2ec7-d1c0-41a0-94ed-73650e9e14f3',
 'Name: DAVENTRY SEN, Project Number: 24445600, ID: 05d8f26a-a0ed-48b8-9473-2fd560b40b41',
 'Name: GEORGE MITCHELL SCHOOL, Project Number: 24427900, ID: 02c7f3f6-88f1-4e22-bca5-97805c254665',
 'Name: LEEDS UTC, Project Number: 24404800, ID: 7c60f84d-ef45-4a45-bad1-80ff6e032611',
 'Name: SHEFFIELD UTC 2 - CBDM, Project Number: 24283800, ID: 5e8e203d-e613-47e0-83c8-085ba22eebd2',
 'Name: BAM - NISHKAM SCHOOL LONDON, Project Number: 24268900, ID: 37ab8156-4595-4ac9-bbd3-d4793c8ef1a2',
 'Name: BRIXTON HILL REDEVELOPMENT, Project Number: 24223600, ID: ea333a93-0a07-4da3-b134-aa9325153791',
 'Name: BAM PSBP RUTH GORSE ACADEMY, Project Number: 24015300, ID: 905b716b-981d-4d96-82f7-503c55b8a49d',
 'Name: BATCH 53 - PSBP HILLINGDON & H, Project Number: 23933500, ID: f40728e7-d1a0-452a-abd4-ab84143b4ce5',
 'Name: INGLEBY BARWICK FREE SCHOOL, Project Number: 23878400, ID: cee75cc9-b70c-45b

### Storing and Loading DDB Types

To prevent constantly querying the server for information about parameter types, asset types, or other similar information, we can download all of this information and load it as necessary.  This is much quicker, and there are methods to query by id or by name.

The following command will download all types, convert them to objects, then pickle them into a local file that you can quickly load as necessary.

In [4]:
asyncio.run(pyddb.regenerate_all_types())

In the following example, we're loading these types back in, showing off the different ways they can be loaded.

In [5]:
async def main():
    
    # you can get a single parameter by name or id:
    area = pyddb.get_parameter_type_by_name("Area")
    volume = pyddb.get_parameter_type_by_id("fdb85750-6c8a-4033-94e4-d91d5825e788")
    
    print(area)
    print(volume)
    
asyncio.run(main())

Name: Area
Name: Volume


In [6]:
async def main():
    
    # you can get parameters by name or id in a single API call:
    area, volume = pyddb.get_parameter_types_by_name(["Area", "Volume"])
    
    print(area)
    print(volume)
    
asyncio.run(main())

Name: Area
Name: Volume


All of this can be applied to parameter types, asset types, source types, tags, or units:

In [7]:
async def main():
    
    asset_type_site = pyddb.get_asset_type_by_name("site")
    source_type_derived_value = pyddb.get_source_type_by_name("Derived Value")
    tag_cladding = pyddb.get_tag_by_name("Cladding")
    unit_m3 = pyddb.get_unit_by_name("m³")
    
    print(asset_type_site)
    print(source_type_derived_value)
    print(tag_cladding)
    print(unit_m3)
    
asyncio.run(main())

Name: site
Name: Derived Value
Name: Cladding, Tag Type: Name: Parameter Category
Name: m³


### Getting and Posting Projects

A posted project will return the project even if it already exists, much like the get project method.

When getting projects, the search keyword can be used to retreive a list of projects by job number, job name, project manager, or project director.

Note that the keyword 'page_limit' should be entered if you want to retreive over 100 projects.

In [8]:
async def main():
    ddb = pyddb.DDB(url=pyddb.BaseURL.sandbox)
    
    post_project = await ddb.post_project(project_number="21515700")
    [get_project] = await ddb.get_projects(search="21515700")
    
    print(post_project)
    print(get_project)
    
asyncio.run(main())

Getting existing project...
Name: CROSSRAIL SURFACE - DESK STUDY, Project Number: 21515700
Name: CROSSRAIL SURFACE - DESK STUDY, Project Number: 21515700


### Getting and Posting Assets


#### Getting Assets
To get assets, we can call the method at DDB, Project, or Asset level to get all assets within the environment, project, or children of a particular asset.

There are also other arguments that can be passed in such as 'asset_type_id' or 'descendants_of' to filter to a specific asset type or all assets below a given asset in the tree.

In [9]:
async def main():
    
    ddb = pyddb.DDB(url=pyddb.BaseURL.sandbox)
    
    [project] = await ddb.get_projects(search="21515700")
    
    # ddb_sandbox_assets = await ddb.get_assets()
    project_sites = await project.get_assets(asset_type_id = pyddb.get_asset_type_by_name("site").id)
    site_child_assets = await project_sites[0].get_assets()

    # print(ddb_sandbox_assets)
    print(project_sites)
    print(site_child_assets)
asyncio.run(main())

['Name: Chain 23w43e, Type: site, ID: 4cdd0190-3018-4051-954c-defed5c723c6', 'Name: Chain 23w3, Type: site, ID: a403543d-2065-4edd-9afa-d9038ad179a2', 'Name: Site 1, Type: site, ID: 91b7314f-1707-4d71-9cbf-610cb1e9cf81', 'Name: Chain 23w43e2, Type: site, ID: cb13bb89-48ad-490d-b853-b992d965120b', 'Name: Chain 23w43e22, Type: site, ID: 21676a57-75e2-4e36-abc4-0275658c619c', 'Name: Chain 23w43e224, Type: site, ID: ed3b5313-3f31-4c3c-b48d-203c1b8c58e9', 'Name: Chain 23w43e2f, Type: site, ID: b7cc3909-e543-4514-b785-7074a93fab1a', 'Name: Chain 23w43, Type: site, ID: ecb79137-4e09-4010-8c69-bac295b78027', 'Name: My DDB Site, Type: site, ID: 82762bfc-cd3a-4667-9ffe-8e58318f19cd']
['Name: Chain 2, Type: building, ID: c041019d-9321-4541-a1de-8bbafcdb3f6f']


#### Posting Asset Trees
To post many assets in a tree structure, you can define each asset as shown below.

The `parent` parameter must be an `Asset` or `NewAsset` object, and can refer to an existing asset or a new one.
If you specify a new asset as a parent while one already exists in its place, it will update the existing asset instead.

Note that in the following example I am creating the variable and assigning it during this expression using the `:=` operator.

In [10]:
async def main():

    ddb = pyddb.DDB(url=pyddb.BaseURL.sandbox)

    project = await ddb.post_project(project_number="21515700")

    new_assets = [
        my_site := pyddb.NewAsset(
            asset_type=pyddb.get_asset_type_by_name("site"),
            name="My DDB Site",
            parent=None,
        ),
        my_building := pyddb.NewAsset(
            asset_type=pyddb.get_asset_type_by_name("building"),
            name="My DDB Building",
            parent=my_site,
        ),
        my_space_type := pyddb.NewAsset(
            asset_type=pyddb.get_asset_type_by_name("space type"),
            name="MY DDB Space Type",
            parent=my_building,
        ),
        my_first_space_instance := pyddb.NewAsset(
            asset_type=pyddb.get_asset_type_by_name("space instance"),
            name="My First DDB Space Instance",
            parent=my_space_type,
        ),
        my_second_space_instance := pyddb.NewAsset(
            asset_type=pyddb.get_asset_type_by_name("space instance"),
            name="My Second DDB Space Instance",
            parent=my_space_type,
        ),
    ]
    newly_posted_assets = await project.post_assets(assets=new_assets)
    print(newly_posted_assets)


asyncio.run(main())


Getting existing project...
['Name: My DDB Site, Type: site, ID: 82762bfc-cd3a-4667-9ffe-8e58318f19cd', 'Name: My DDB Building, Type: building, ID: 019de055-53f4-43f2-bf9a-43d1bca5f196', 'Name: MY DDB Space Type, Type: space type, ID: 4fafb692-36d5-4e6a-9692-7983046d993f', 'Name: My First DDB Space Instance, Type: space instance, ID: 2110b107-cad5-44a4-acd1-851020dcd24f', 'Name: My Second DDB Space Instance, Type: space instance, ID: e824321d-b5c3-4139-8e50-1cddef360685', 'Name: My DDB Site, Type: site, ID: 82762bfc-cd3a-4667-9ffe-8e58318f19cd', 'Name: My DDB Building, Type: building, ID: 019de055-53f4-43f2-bf9a-43d1bca5f196', 'Name: MY DDB Space Type, Type: space type, ID: 4fafb692-36d5-4e6a-9692-7983046d993f', 'Name: My First DDB Space Instance, Type: space instance, ID: 2110b107-cad5-44a4-acd1-851020dcd24f', 'Name: My Second DDB Space Instance, Type: space instance, ID: e824321d-b5c3-4139-8e50-1cddef360685']


### Getting and Posting Sources

#### Getting Sources
To get sources, we can call the method at DDB, or Project level to get all sources within the environment or project.

There are also other arguments that can be passed in such as 'source_type_id', 'title', or 'reference' to filter to a specific source type or all sources matching other criteria.

In [11]:
async def main():
    
    ddb = pyddb.DDB(url=pyddb.BaseURL.sandbox)
    
    [project] = await ddb.get_projects(search="21515700")
    
    # ddb_sandbox_sources = await ddb.get_sources()
    project_sources = await project.get_sources()

    # print(ddb_sandbox_sources)
    print(project_sources)
asyncio.run(main())

['Title: Schools Daylighting Spreadsheet, Reference: Schools Daylighting Spreadsheet, Source Type: Project Document, ID: e8fdac47-9656-47eb-b29f-455d88209758', 'Title: Schools Daylighting Spreadsheet, Reference: Schools Daylighting Spreadsheet, Source Type: Project Document, ID: 979da291-6de2-4db4-840a-22309c45d50c', 'Title: Schools Daylighting Spreadsheet, Reference: Schools Daylighting Spreadsheet, Source Type: Project Document, ID: f094eda0-9982-4491-8337-44b7b1fe03f7', 'Title: Schools Daylighting Spreadsheet, Reference: Schools Daylighting Spreadsheet, Source Type: Project Document, ID: 3a546841-5da2-4029-a100-94ff3862fe8a', 'Title: Schools Daylighting Spreadsheet, Reference: Schools Daylighting Spreadsheet, Source Type: Project Document, ID: 10da0fb9-679e-47e9-ae42-d503d4a42990', 'Title: Schools Daylighting Spreadsheet, Reference: Schools Daylighting Spreadsheet, Source Type: Project Document, ID: b9de4aae-88cf-498e-88e4-be821a3346a5', 'Title: Schools Daylighting Spreadsheet, Refe

#### Posting Sources
To post multiple sources, we can create a list of NewSource objects as shown below and use the Project method to pose them.

If you specify a source that already exists, no duplicate will be created.

This method will also return these back as Source objects.

In [12]:
async def main():

    ddb = pyddb.DDB(url=pyddb.BaseURL.sandbox)

    [project] = await ddb.get_projects(search="21515700")
    [new_source] = await project.post_sources(
        sources=[
            pyddb.NewSource(
                source_type=pyddb.get_source_type_by_name("Derived Value"),
                title="My source title",
                reference="My source reference",
            )
        ]
    )
    print(new_source)


asyncio.run(main())


Title: My source title, Reference: My source reference, Source Type: Derived Value


### Getting and Posting Parameters

#### Getting Parameters
To get parameters, we can call the method at DDB, Project, or Asset level to get all sources within the environment or project.

There are also other arguments that can be passed in such as 'source_type_id', 'title', or 'reference' to filter to a specific source type or all sources matching other criteria.

In [13]:
async def main():
    
    ddb = pyddb.DDB(url=pyddb.BaseURL.sandbox)
    
    [project] = await ddb.get_projects(search="21515700")
    
    # ddb_sandbox_parameters = await ddb.get_parameters()
    project_parameters = await project.get_parameters()

    # print(ddb_sandbox_parameters)
    print(project_parameters)
asyncio.run(main())

['Parameter Type: Name: VIP system required, Revision: "Value: \'Value: True, Unit: None, ID: 58c06155-9ed0-c71e-708d-499cc5e1b048\', Source: \'Title: Assumption, Reference: Assumption, Source Type: Assumption, ID: 3c9a44e3-be3e-4aed-842e-6f6fb05632ad\', Status: answered, ID: ab6ab684-03a4-4167-9530-a48932560935", ID: 1a42045e-e9cc-4f16-a893-fe656443eb18', 'Parameter Type: Name: Unit description, Revision: "Value: \'Value: A nice office, Unit: None, ID: dcc06155-9ef1-3073-5b99-ef22939e4479\', Source: \'Title: Assumption, Reference: Assumption, Source Type: Assumption, ID: 3c9a44e3-be3e-4aed-842e-6f6fb05632ad\', Status: answered, ID: c0c82c0a-fdf1-496d-a885-efb0b5877308", ID: f9cf6fe9-cc7b-47ad-956e-a9228cd7728f', 'Parameter Type: Name: Type of apartment units, Revision: "Value: \'Value: Studio, Unit: None, ID: 1cc06155-9e9b-84f3-66da-f841fde3cf02\', Source: \'Title: Assumption, Reference: Assumption, Source Type: Assumption, ID: 3c9a44e3-be3e-4aed-842e-6f6fb05632ad\', Status: answered,

### Posting Parameters

To post parameters, you will first need an asset or project to assign them to.  If you're posting them with revisions (values), you'll also need a source for the revision.

You can get these assets or sources as shown previously.  This is what I've done in the example below.

I've defined a list of NewParameter objects, with NewRevision objects as their revision properties.

These can be posted as DDB level, but don't curently return Parameter objects back.  I might add this in the future.

In [14]:
async def main():

    ddb = pyddb.DDB(url=pyddb.BaseURL.sandbox)

    [project] = await ddb.get_projects(search="21515700")

    """posting a new source"""
    [new_source] = await project.post_sources(
        sources=[
            pyddb.NewSource(
                source_type=pyddb.get_source_type_by_name("Derived Value"),
                title="My source title",
                reference="My source reference",
            )
        ]
    )
    """posting new assets"""
    new_assets = [
        my_site := pyddb.NewAsset(
            asset_type=pyddb.get_asset_type_by_name("site"),
            name="My DDB Site",
            parent=None,
        ),
        my_building := pyddb.NewAsset(
            asset_type=pyddb.get_asset_type_by_name("building"),
            name="My DDB Building",
            parent=my_site,
        ),
        my_space_type := pyddb.NewAsset(
            asset_type=pyddb.get_asset_type_by_name("space type"),
            name="MY DDB Space Type",
            parent=my_building,
        ),
        my_first_space_instance := pyddb.NewAsset(
            asset_type=pyddb.get_asset_type_by_name("space instance"),
            name="My First DDB Space Instance",
            parent=my_space_type,
        ),
        my_second_space_instance := pyddb.NewAsset(
            asset_type=pyddb.get_asset_type_by_name("space instance"),
            name="My Second DDB Space Instance",
            parent=my_space_type,
        ),
    ]

    """selecting an asset"""
    newly_posted_assets = await project.post_assets(assets=new_assets)
    my_newly_post_building = next(
        x for x in newly_posted_assets if x.name == "My First DDB Space Instance"
    )

    """posting new parameters"""
    new_parameters = [
        pyddb.NewParameter(
            parameter_type=pyddb.get_parameter_type_by_name("Area"),
            revision=pyddb.NewRevision(
                value=20, unit=pyddb.get_unit_by_name("m²"), source=new_source
            ),
            parent=my_newly_post_building,
        )
    ]

    await ddb.post_parameters(project=project, parameters=new_parameters)


asyncio.run(main())


### Getting and Posting Tags

#### Getting Tags
To get parameters, we can call the method at DDB, Project, or Asset level to get all sources within the environment or project.

There are also other arguments that can be passed in such as 'source_type_id', 'title', or 'reference' to filter to a specific source type or all sources matching other criteria.

#### Posting Tags
To get parameters, we can call the method at DDB, Project, or Asset level to get all sources within the environment or project.

There are also other arguments that can be passed in such as 'source_type_id', 'title', or 'reference' to filter to a specific source type or all sources matching other criteria.